In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [993 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,257 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [966 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,235 kB]
Hit:13 https://ppa.launchpadcon

In [5]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, to_date
import time
from datetime import datetime

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [8]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)
df = df.withColumn("year", year(df["date"]))
df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|2022|
+--------------------+----------+-------

In [10]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")


In [15]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("SELECT year, round(avg(price),2) as avg_price FROM home_sales WHERE bedrooms=4 GROUP BY year").show()


+----+---------+
|year|avg_price|
+----+---------+
|2022|296363.88|
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
+----+---------+



In [16]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("SELECT date_built, round(avg(price),2) as avg_price FROM home_sales WHERE bedrooms=3 AND bathrooms=3 GROUP BY date_built").show()



+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2015| 288770.3|
|      2013|295962.27|
|      2014|290852.27|
|      2012|293683.19|
|      2016|290555.07|
|      2010|292859.62|
|      2011|291117.47|
|      2017|292676.79|
+----------+---------+



In [18]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""
  SELECT date_built, round(avg(price),2) as avg_price
  FROM home_sales
  WHERE bedrooms=3
  AND bathrooms=3
  AND floors=2
  AND sqft_living>=2000
  GROUP BY date_built
  """).show()



+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2015|297609.97|
|      2013|303676.79|
|      2014|298264.72|
|      2012|307539.97|
|      2016| 293965.1|
|      2010|285010.22|
|      2011|276553.81|
|      2017|280317.58|
+----------+---------+



In [22]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
spark.sql("""
  SELECT view, round(avg(price),2) as avg_price
  FROM home_sales
  WHERE price>=350000
  GROUP BY view
  """).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

--- 0.5306456089019775 seconds ---


In [20]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [21]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [24]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
spark.sql("""
  SELECT view, round(avg(price),2) as avg_price
  FROM home_sales
  WHERE price>=350000
  GROUP BY view
  """).show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

--- 0.5777051448822021 seconds ---


On my machine the uncached version ran .04 seconds faster

In [25]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [26]:
# 11. Read the parquet formatted data.
p_df = spark.read.parquet("home_sales_partitioned")

In [27]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView("home_sales_p")

In [28]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()
spark.sql("""
  SELECT view, round(avg(price),2) as avg_price
  FROM home_sales_p
  WHERE price>=350000
  GROUP BY view
  """).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

--- 0.8991541862487793 seconds ---


The partitioned parquet version ran .32 seconds slower

In [29]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

DataFrame[]

In [30]:
# 15. Check if the home_sales is no longer cached

if spark.catalog.isCached("home_sales"):
  print("A table is still cached.")
else:
  print("The table has been uncached.")

The table has been uncached.
